In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from matplotlib.transforms import IdentityTransform, TransformedBbox, Bbox
from scipy.optimize import brute, fmin

from deepwalk.models import negative_sampling
from deepwalk.walks import construct_co_occurrence_matrix

In [ ]:
# Class used to annotate plots.
class AngleAnnotation(Arc):
    """
    Draws an arc between two vectors which appears circular in display space.
    """
    def __init__(self, xy, p1, p2, size=75, unit="points", ax=None,
                 text="", textposition="inside", text_kw=None, **kwargs):
        """
        Parameters
        ----------
        xy, p1, p2 : tuple or array of two floats
            Center position and two points. Angle annotation is drawn between
            the two vectors connecting *p1* and *p2* with *xy*, respectively.
            Units are data coordinates.

        size : float
            Diameter of the angle annotation in units specified by *unit*.

        unit : str
            One of the following strings to specify the unit of *size*:

            * "pixels": pixels
            * "points": points, use points instead of pixels to not have a
              dependence on the DPI
            * "axes width", "axes height": relative units of Axes width, height
            * "axes min", "axes max": minimum or maximum of relative Axes
              width, height

        ax : `matplotlib.axes.Axes`
            The Axes to add the angle annotation to.

        text : str
            The text to mark the angle with.

        textposition : {"inside", "outside", "edge"}
            Whether to show the text in- or outside the arc. "edge" can be used
            for custom positions anchored at the arc's edge.

        text_kw : dict
            Dictionary of arguments passed to the Annotation.

        **kwargs
            Further parameters are passed to `matplotlib.patches.Arc`. Use this
            to specify, color, linewidth etc. of the arc.

        """
        self.ax = ax or plt.gca()
        self._xydata = xy  # in data coordinates
        self.vec1 = p1
        self.vec2 = p2
        self.size = size
        self.unit = unit
        self.textposition = textposition

        super().__init__(self._xydata, size, size, angle=0.0,
                         theta1=self.theta1, theta2=self.theta2, **kwargs)

        self.set_transform(IdentityTransform())
        self.ax.add_patch(self)

        self.kw = dict(ha="center", va="center",
                       xycoords=IdentityTransform(),
                       xytext=(0, 0), textcoords="offset points",
                       annotation_clip=True)
        self.kw.update(text_kw or {})
        self.text = ax.annotate(text, xy=self._center, **self.kw)

    def get_size(self):
        factor = 1.
        if self.unit == "points":
            factor = self.ax.figure.dpi / 72.
        elif self.unit[:4] == "axes":
            b = TransformedBbox(Bbox.from_bounds(0, 0, 1, 1),
                                self.ax.transAxes)
            dic = {"max": max(b.width, b.height),
                   "min": min(b.width, b.height),
                   "width": b.width, "height": b.height}
            factor = dic[self.unit[5:]]
        return self.size * factor

    def set_size(self, size):
        self.size = size

    def get_center_in_pixels(self):
        """return center in pixels"""
        return self.ax.transData.transform(self._xydata)

    def set_center(self, xy):
        """set center in data coordinates"""
        self._xydata = xy

    def get_theta(self, vec):
        vec_in_pixels = self.ax.transData.transform(vec) - self._center
        return np.rad2deg(np.arctan2(vec_in_pixels[1], vec_in_pixels[0]))

    def get_theta1(self):
        return self.get_theta(self.vec1)

    def get_theta2(self):
        return self.get_theta(self.vec2)

    def set_theta(self, angle):
        pass

    # Redefine attributes of the Arc to always give values in pixel space
    _center = property(get_center_in_pixels, set_center)
    theta1 = property(get_theta1, set_theta)
    theta2 = property(get_theta2, set_theta)
    width = property(get_size, set_size)
    height = property(get_size, set_size)

    # The following two methods are needed to update the text position.
    def draw(self, renderer):
        self.update_text()
        super().draw(renderer)

    def update_text(self):
        c = self._center
        s = self.get_size()
        angle_span = (self.theta2 - self.theta1) % 360
        angle = np.deg2rad(self.theta1 + angle_span / 2)
        r = s / 2
        if self.textposition == "inside":
            r = s / np.interp(angle_span, [60, 90, 135, 180],
                                          [3.3, 3.5, 3.8, 4])
        self.text.xy = c + r * np.array([np.cos(angle), np.sin(angle)])
        if self.textposition == "outside":
            def R90(a, r, w, h):
                if a < np.arctan(h/2/(r+w/2)):
                    return np.sqrt((r+w/2)**2 + (np.tan(a)*(r+w/2))**2)
                else:
                    c = np.sqrt((w/2)**2+(h/2)**2)
                    T = np.arcsin(c * np.cos(np.pi/2 - a + np.arcsin(h/2/c))/r)
                    xy = r * np.array([np.cos(a + T), np.sin(a + T)])
                    xy += np.array([w/2, h/2])
                    return np.sqrt(np.sum(xy**2))

            def R(a, r, w, h):
                aa = (a % (np.pi/4))*((a % (np.pi/2)) <= np.pi/4) + \
                     (np.pi/4 - (a % (np.pi/4)))*((a % (np.pi/2)) >= np.pi/4)
                return R90(aa, r, *[w, h][::int(np.sign(np.cos(2*a)))])

            bbox = self.text.get_window_extent()
            X = R(angle, r, bbox.width, bbox.height)
            trans = self.ax.figure.dpi_scale_trans.inverted()
            offs = trans.transform(((X-s/2), 0))[0] * 72
            self.text.set_position([offs*np.cos(angle), offs*np.sin(angle)])

# Brute Force $K=2$ Clusters

In [ ]:
def loss(z, *params):
    angle1, angle2 = z
    a, b, y1, y2 = params
    
    x1 = np.array([np.cos(np.deg2rad(angle1)), np.sin(np.deg2rad(angle1))])
    x2 = np.array([np.cos(np.deg2rad(angle2)), np.sin(np.deg2rad(angle2))])
    
    z1 = np.exp(np.dot(x1,y1)) + np.exp(np.dot(x1,y2))
    z2 = np.exp(np.dot(x2,y1)) + np.exp(np.dot(x2,y2))
    
    s1 = a*np.log(np.exp(np.dot(x1,y1))/z1) + b*np.log(np.exp(np.dot(x1,y2))/z1)
    s2 = b*np.log(np.exp(np.dot(x2,y1))/z2) + a*np.log(np.exp(np.dot(x2,y2))/z2)
    
    l = -(s1 + s2)
    return l


y1 = np.array([1,0])
b = 0.1
a = 8*b

x = np.arange(0, 181, 1)
theta = slice(0, 181, 1)
rranges = (theta, theta)

best_loss = 1000
best_xangles = None
best_y2_angle = None
y2v = x
for i in range(len(x)):
    y2 = np.array([np.cos(np.deg2rad(y2v[i])), np.sin(np.deg2rad(y2v[i]))])
    
    params = (a, b, y1, y2)
    resbrute = brute(loss, rranges, args=params, full_output=True, finish=fmin)
    print(i, y2v[i], resbrute[1], resbrute[0])
    if resbrute[1] < best_loss:
        best_loss = resbrute[1]
        best_xangles = resbrute[0]
        best_y2_angle = y2v[i]

In [ ]:
print(best_loss, best_xangles, best_y2_angle)

In [ ]:
# Plot embeddings

# Node embeddings
fig, ax = plt.subplots(1,1, figsize=(4,4))
#ax.scatter(np.cos(np.deg2rad(np.arange(0, 361, 2))), np.sin(np.deg2rad(np.arange(0, 361, 2))), marker='.', linewidths=.00001, alpha=0.7)
ax.plot(np.cos(np.deg2rad(np.arange(0, 361, 5))), np.sin(np.deg2rad(np.arange(0, 361, 5))), linestyle='--', linewidth=1)
ax.scatter(0,0,color='black')
ax.quiver([0], [0], np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0)), units='xy', angles='xy', scale_units='xy', scale=1, color='r')
ax.quiver([0], [0], np.cos(np.deg2rad(180)), np.sin(np.deg2rad(180)), units='xy', angles='xy', scale_units='xy', scale=1, color='b')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), 
                     np.array([np.cos(np.deg2rad(180)), np.sin(np.deg2rad(180))]), ax=ax, size=75, 
                    text=f"{180.0:.2f}",
                    textposition='edge')
ax.set(xlim=(-1, 1), ylim=(-1, 1), xticks=np.linspace(-1, 1, 5), yticks=np.linspace(-1, 1, 5))
fig.tight_layout()
fig.savefig('brute_k2_x.png', bbox_inches='tight', bbox_pad=0)
plt.show()

# Context embeddings
fig, ax = plt.subplots(1,1, figsize=(4,4))
#ax.scatter(np.cos(np.deg2rad(np.arange(0, 361, 2))), np.sin(np.deg2rad(np.arange(0, 361, 2))), marker='.', linewidths=.00001, alpha=0.7)
ax.plot(np.cos(np.deg2rad(np.arange(0, 361, 5))), np.sin(np.deg2rad(np.arange(0, 361, 5))), linestyle='--', linewidth=1)
ax.scatter(0,0,color='black')
ax.quiver([0], [0], 1, 0, units='xy', angles='xy', scale_units='xy', scale=1, color='r')
ax.quiver([0], [0], np.cos(np.deg2rad(180)), np.sin(np.deg2rad(180)), units='xy', angles='xy', scale_units='xy', scale=1, color='b')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), 
                     np.array([np.cos(np.deg2rad(180)), np.sin(np.deg2rad(180))]), ax=ax, size=75, 
                    text=f"{180.0:.2f}",
                    textposition='edge')
ax.set(xlim=(-1, 1), ylim=(-1, 1), xticks=np.linspace(-1, 1, 5), yticks=np.linspace(-1, 1, 5))
fig.tight_layout()
fig.savefig('brute_k2_y.png', bbox_inches='tight', bbox_pad=0)
plt.show()

# Brute Force $k=3$ Clusters

In [ ]:
def loss(z, *params):
    angle1, angle2, angle3 = z
    a, b, y1, y2, y3 = params
    
    x1 = np.array([np.cos(np.deg2rad(angle1)), np.sin(np.deg2rad(angle1))])
    x2 = np.array([np.cos(np.deg2rad(angle2)), np.sin(np.deg2rad(angle2))])
    x3 = np.array([np.cos(np.deg2rad(angle3)), np.sin(np.deg2rad(angle3))])
    
    z1 = np.exp(np.dot(x1,y1)) + np.exp(np.dot(x1,y2)) + np.exp(np.dot(x1,y3))
    z2 = np.exp(np.dot(x2,y1)) + np.exp(np.dot(x2,y2)) + np.exp(np.dot(x2,y3))
    z3 = np.exp(np.dot(x3,y1)) + np.exp(np.dot(x3,y2)) + np.exp(np.dot(x3,y3))
    
    s1 = a*np.log(np.exp(np.dot(x1,y1))/z1) + b*np.log(np.exp(np.dot(x1,y2))/z1) + b*np.log(np.exp(np.dot(x1,y3))/z1)
    s2 = b*np.log(np.exp(np.dot(x2,y1))/z2) + a*np.log(np.exp(np.dot(x2,y2))/z2) + b*np.log(np.exp(np.dot(x2,y3))/z2)
    s3 = b*np.log(np.exp(np.dot(x3,y1))/z3) + b*np.log(np.exp(np.dot(x3,y2))/z3) + a*np.log(np.exp(np.dot(x3,y3))/z3)
    
    l = -(s1 + s2 + s3)
    return l


y1 = np.array([1,0])
b = 0.1
a = 8*b

x = np.arange(0, 185, 5)
theta = slice(0, 185, 5)
rranges = (theta, theta, theta)

best_loss = 1000
best_xangles = None
best_y2_angle = None
best_y3_angle = None
y2v, y3v = np.meshgrid(x, x, indexing='ij')
for i in range(len(x)):
    for j in range(len(x)):
        y2 = np.array([np.cos(np.deg2rad(y2v[i,j])), np.sin(np.deg2rad(y2v[i,j]))])
        y3 = np.array([np.cos(np.deg2rad(y3v[i,j])), -np.sin(np.deg2rad(y3v[i,j]))])
    
        params = (a, b, y1, y2, y3)
        resbrute = brute(loss, rranges, args=params, full_output=True, finish=fmin)
        print(i, j , y2v[i,j], y3v[i,j], resbrute[1], resbrute[0])
        if resbrute[1] < best_loss:
            best_loss = resbrute[1]
            best_xangles = resbrute[0]
            best_y2_angle = y2v[i,j]
            best_y3_angle = -y3v[i,j]
            #print(best_loss, best_xangles, best_y2_angle, best_y3_angle)

In [ ]:
print(best_loss, best_xangles, best_y2_angle, best_y3_angle)

In [ ]:
# plot node embeddings
fig, ax = plt.subplots(1,1, figsize=(4,4))
#ax.scatter(np.cos(np.deg2rad(np.arange(0, 361, 2))), np.sin(np.deg2rad(np.arange(0, 361, 2))), marker='.', linewidths=.00001, alpha=0.7)
ax.plot(np.cos(np.deg2rad(np.arange(0, 361, 5))), np.sin(np.deg2rad(np.arange(0, 361, 5))), linestyle='--', linewidth=1)
ax.scatter(0,0,color='black')
ax.quiver([0], [0], np.cos(np.deg2rad(-1.18453764e-03)), np.sin(np.deg2rad(-1.18453764e-03)), units='xy', angles='xy', scale_units='xy', scale=1, color='r')
ax.quiver([0], [0], np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120)), units='xy', angles='xy', scale_units='xy', scale=1, color='b')
ax.quiver([0], [0], np.cos(np.deg2rad(240)), np.sin(np.deg2rad(240)), units='xy', angles='xy', scale_units='xy', scale=1, color='g')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), 
                     np.array([np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120))]), 
                     np.array([np.cos(np.deg2rad(240)), np.sin(np.deg2rad(240))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(240)), np.sin(np.deg2rad(240))]), 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
ax.set(xlim=(-1, 1), ylim=(-1, 1), xticks=np.linspace(-1, 1, 5), yticks=np.linspace(-1, 1, 5))
fig.tight_layout()
fig.savefig('brute_k3_x.png', bbox_inches='tight', bbox_pad=0)
plt.show()

# Plot Context embeddings
fig, ax = plt.subplots(1,1, figsize=(4,4))
#ax.scatter(np.cos(np.deg2rad(np.arange(0, 361, 2))), np.sin(np.deg2rad(np.arange(0, 361, 2))), marker='.', linewidths=.00001, alpha=0.7)
ax.plot(np.cos(np.deg2rad(np.arange(0, 361, 5))), np.sin(np.deg2rad(np.arange(0, 361, 5))), linestyle='--', linewidth=1)
ax.scatter(0,0,color='black')
ax.quiver([0], [0], 1, 0, units='xy', angles='xy', scale_units='xy', scale=1, color='r')
ax.quiver([0], [0], np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120)), units='xy', angles='xy', scale_units='xy', scale=1, color='b')
ax.quiver([0], [0], np.cos(np.deg2rad(-120)), np.sin(np.deg2rad(-120)), units='xy', angles='xy', scale_units='xy', scale=1, color='g')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), 
                     np.array([np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(120)), np.sin(np.deg2rad(120))]), 
                     np.array([np.cos(np.deg2rad(240)), np.sin(np.deg2rad(240))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
am = AngleAnnotation([0,0], 
                     np.array([np.cos(np.deg2rad(240)), np.sin(np.deg2rad(240))]), 
                     np.array([np.cos(np.deg2rad(0)), np.sin(np.deg2rad(0))]), ax=ax, size=75, 
                    text=f"{120.0:.2f}",
                    textposition='edge')
ax.set(xlim=(-1, 1), ylim=(-1, 1), xticks=np.linspace(-1, 1, 5), yticks=np.linspace(-1, 1, 5))
fig.tight_layout()
fig.savefig('brute_k3_y.png', bbox_inches='tight', bbox_pad=0)
plt.show()